<a href="https://colab.research.google.com/github/falvarezamarillo/gobierno_datos_207/blob/main/Indicadores_calidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install newsapi-python
!pip install pandas numpy requests

In [2]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
from newsapi import NewsApiClient

# -------------------------------
# 1. Configuración de API Keys
# -------------------------------
# API Key para NewsAPI (regístrate en https://newsapi.org/)
NEWS_API_KEY = "f89e41ed7e8f4b66a0a0279304f2c563"  # <-- Reemplaza aquí

# API Key para OpenWeatherMap (regístrate en https://openweathermap.org/)
OPENWEATHER_API_KEY = "9dfdfa17227e85b405605d9ec193d242"  # <-- Reemplaza aquí

# Nota: La API del BCRP no requiere API key para consultar las series.

# -------------------------------

In [5]:
# 2. Extracción de Datos Externos
# -------------------------------

## a) Noticias de Perú usando fuentes específicas de NewsAPI con fallback a get_everything
## a) Noticias de Perú usando get_everything
def fetch_news_data():
    # Inicializa el cliente de NewsAPI
    newsapi = NewsApiClient(api_key=NEWS_API_KEY)

    # Consulta todas las noticias que contengan "peru" en español
    response = newsapi.get_everything(q="peru", language="es")
    articles = response.get("articles", [])

    # Extrae la información relevante y crea una lista de registros
    news_list = []
    for art in articles:
        news_list.append({
            "titulo": art.get("title"),
            "descripcion": art.get("description"),
            "fecha": art.get("publishedAt"),
            "fuente": art.get("source", {}).get("name")
        })
    return pd.DataFrame(news_list)

df_news = fetch_news_data()



In [6]:
## b) Temperatura en Perú por Ciudad usando OpenWeatherMap
def fetch_temperature_data(city):
    url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": f"{city},pe",  # Ciudad en Perú
        "appid": OPENWEATHER_API_KEY,
        "units": "metric",  # Temperatura en Celsius
        "lang": "es"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        temp_data = {
            "ciudad": city,
            "temperatura": data.get("main", {}).get("temp"),
            "humedad": data.get("main", {}).get("humidity"),
            "fecha": datetime.utcfromtimestamp(data.get("dt")).strftime("%Y-%m-%d %H:%M:%S")
        }
        return temp_data
    else:
        print(f"Error al obtener datos para {city}: {response.status_code}")
        return None

# Lista de las capitales de los departamentos de Perú
capitales = [
    "Chachapoyas", "Huaraz", "Abancay", "Arequipa", "Ayacucho", "Cajamarca", "Callao",
    "Cusco", "Huancavelica", "Huánuco", "Ica", "Huancayo", "Trujillo", "Chiclayo",
    "Lima", "Iquitos", "Puerto Maldonado", "Moquegua", "Cerro de Pasco", "Piura",
    "Puno", "Moyobamba", "Tacna", "Tumbes", "Pucallpa"
]
temp_data_list = [fetch_temperature_data(capital) for capital in capitales]
temp_data_list = [d for d in temp_data_list if d is not None]
df_temp = pd.DataFrame(temp_data_list)

In [16]:
## c) Tipo de Cambio: Variaciones del Día en Perú de los Últimos 4 Años usando la API del BCRP
# Se utiliza el código de serie "3186" (ajústalo si es necesario) según la documentación:
# https://estadisticas.bcrp.gob.pe/estadisticas/series/ayuda/api
def fetch_exchange_rate_data(series_id="PD04637PD"):
    # Calcular fechas para los últimos 4 años
    fecha_final = datetime.today()
    fecha_inicial = fecha_final - timedelta(days=4*365)  # Aproximación de 4 años

    # Formato de fecha requerido: 'dd-mm-yyyy'
    fecha_inicial_str = fecha_inicial.strftime("%d-%m-%Y")
    fecha_final_str = fecha_final.strftime("%d-%m-%Y")

    # Construir la URL según la documentación del BCRP
    url = f"https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{series_id}/json/" #{fecha_inicial_str}/{fecha_final_str}

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        # Se espera que la respuesta contenga una lista de observaciones en el campo "Serie"
        series_data = data.get("Serie", [])
        records = []
        for item in series_data:
            records.append({
                "fecha": item.get("fecha"),
                "tipo_cambio": float(item.get("dato", 0))
            })
        return pd.DataFrame(records)
    else:
        print("Error al obtener datos del tipo de cambio:", response.status_code)
        return pd.DataFrame()

df_exchange = fetch_exchange_rate_data()

In [17]:
# -------------------------------
# 3. Aplicación de Reglas de Calidad de Datos
# -------------------------------
def quality_report(df, dataset_name="Dataset"):
    report = {}
    total_rows = df.shape[0]

    # Completitud: porcentaje de datos no nulos por columna
    completeness = df.notnull().mean() * 100
    report['completeness'] = completeness.to_dict()

    # Duplicados: cantidad y porcentaje
    duplicate_count = df.duplicated().sum()
    report['duplicates'] = {
        "count": int(duplicate_count),
        "percentage": (duplicate_count / total_rows * 100) if total_rows > 0 else np.nan
    }

    # Validaciones específicas por dataset
    validations = {}
    if dataset_name == "Noticias":
        if "fecha" in df.columns:
            valid_date = pd.to_datetime(df['fecha'], errors='coerce').notnull().mean() * 100
            validations['fecha_valida (%)'] = valid_date
    elif dataset_name == "Temperatura":
        if "temperatura" in df.columns:
            # Suponemos que la temperatura en ciudades de Perú está entre -10 y 50°C
            valid_temp = df['temperatura'].between(-10, 50).mean() * 100
            validations['temperatura_valida (%)'] = valid_temp
    elif dataset_name == "Tipo de Cambio":
        if "tipo_cambio" in df.columns:
            # Se asume que el tipo de cambio USD/PEN está en un rango razonable (por ejemplo, 2.5 a 5.0)
            valid_tc = df['tipo_cambio'].between(2.5, 5.0).mean() * 100
            validations['tipo_cambio_valido (%)'] = valid_tc

    report['validations'] = validations
    report['total_rows'] = total_rows
    return report

# Generar reportes de calidad para cada dataset
report_news = quality_report(df_news, dataset_name="Noticias")
report_temp = quality_report(df_temp, dataset_name="Temperatura")
report_exchange = quality_report(df_exchange, dataset_name="Tipo de Cambio")


# -------------------------------

In [18]:
# 4. Resumen de KPIs de Calidad de Datos
# -------------------------------
def print_quality_summary(report, dataset_name):
    print(f"=== Resumen de Calidad para {dataset_name} ===")
    print(f"Total de registros: {report.get('total_rows', 0)}")
    print("Completitud por columna:")
    for col, pct in report.get('completeness', {}).items():
        print(f"  - {col}: {pct:.2f}%")
    dup = report.get('duplicates', {})
    print(f"Duplicados: {dup.get('count',0)} ({dup.get('percentage',0):.2f}%)")
    print("Validaciones específicas:")
    for key, value in report.get('validations', {}).items():
        print(f"  - {key}: {value:.2f}%")
    print("\n")

print_quality_summary(report_news, "Noticias")
print_quality_summary(report_temp, "Temperatura")
print_quality_summary(report_exchange, "Tipo de Cambio")

=== Resumen de Calidad para Noticias ===
Total de registros: 25
Completitud por columna:
  - titulo: 100.00%
  - descripcion: 100.00%
  - fecha: 100.00%
  - fuente: 100.00%
Duplicados: 0 (0.00%)
Validaciones específicas:
  - fecha_valida (%): 100.00%


=== Resumen de Calidad para Temperatura ===
Total de registros: 25
Completitud por columna:
  - ciudad: 100.00%
  - temperatura: 100.00%
  - humedad: 100.00%
  - fecha: 100.00%
Duplicados: 0 (0.00%)
Validaciones específicas:
  - temperatura_valida (%): 100.00%


=== Resumen de Calidad para Tipo de Cambio ===
Total de registros: 0
Completitud por columna:
Duplicados: 0 (nan%)
Validaciones específicas:


